In [4]:
import tifffile
import functools
import json
import os
import re
from typing import Mapping, Tuple, Union

import click
import numpy as np
from slicedimage import ImageFormat

import starfish.core.util.try_import
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, Features, Number

ROUND_NUM = 1
FOV_NUM = 1
Z_NUM = 366
CHANNEL_NUM = 1

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return tifffile.imread(file_path)

class SampleDataTile(FetchedTile):

    def __init__(self, 
                 file_path: str, 
                 coordinates: Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]],
                 z: int) -> None:
        """
        Parameters
        ----------
        file_path : str
            location of the tile
        coordinates : Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]
            the coordinates for the selected tile, extracted from the metadata
        z : int
            the z-layer for the selected osmFISH tile
        """
        self.file_path = file_path
        self._coordinates = coordinates
        self.z = z
        
    @property
    def shape(self) -> Mapping[Axes, int]:
        raw_shape = self.tile_data().shape
        return {Axes.Y: raw_shape[0], Axes.X: raw_shape[1]}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]:
        return self._coordinates

    def tile_data(self) -> np.ndarray:
        return tifffile.imread(self.file_path)

class SampleDataTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        basename = f"63x_penk_Z000.tif"
        first_file = os.path.join(input_dir, basename)
        
        raw_shape = cached_read_fn(first_file).shape
        
    @property
    def channel_map(self) -> Mapping[str, int]:
        if self.num_c >1:
            return {
                "red": 0,
                "blue": 1,
                "green": 2,
            }
        return 0
    
    def coordinate_map(self, round_: int, z: int):
        # TODO see osmFISH for example on how to dynamically generate this
        
        # dummy coordinates
        return {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
    
    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        basename = f"63x_penk_Z{z+1:03}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = self.coordinate_map(r, z)
        return SampleDataTile(file_path, coordinates, z)

    
def cli(input_dir, output_dir) -> None:
    abs_output_dir = os.path.expanduser(output_dir)
    abs_input_dir = os.path.expanduser(input_dir)
    os.makedirs(abs_output_dir, exist_ok=True)

    primary_tile_fetcher = SampleDataTileFetcher(abs_input_dir)
    primary_image_dimensions: Mapping[Union[str, Axes], int] = {
        Axes.ROUND: ROUND_NUM,
        Axes.CH: CHANNEL_NUM,
        Axes.ZPLANE: Z_NUM,
    }

    write_experiment_json(
        path=output_dir,
        fov_count=FOV_NUM,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        primary_tile_fetcher=primary_tile_fetcher,
        aux_name_to_dimensions={},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    
if __name__ == "__main__":
    cli('sample_data/penk63_raw','sample_data/penk63_formatted')

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x12fee5570> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13140d468> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13140d4c0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13140d570> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13140d5c8> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130f477d8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130f47888> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130f478e0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130f47990> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130f479e8> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130faec50> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130faed00> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130faedb0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130faed58> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x130faee08> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1312e4048> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1312e40a0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1312e40f8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1312e4150> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1312e4200> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13114a468> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13114a518> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13114a570> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13114a620> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x13114a6d0> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1310e8830> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1310e88e0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1310e8990> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1310e89e8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x1310e8a40> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x132ccbba0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x132ccbc50> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x132ccbca8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x132ccbd58> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x132ccbdb0> is a low contrast image
  warn('%s is a low 

In [1]:
from typing import Optional, Tuple
from IPython import get_ipython

import starfish
import starfish.data
from starfish import FieldOfView, IntensityTable

experiment = starfish.Experiment.from_json("/Users/danyfu/dropbox/Starotator/StarFISH/penk63/penk63_formatted/experiment.json")
for key in experiment.keys():
    print(key)

fov_000


In [4]:
ipython = get_ipython()
ipython.magic("gui qt5")

fov0 = experiment['fov_000']
primary_image = fov0.get_image(FieldOfView.PRIMARY_IMAGES)
viewer = starfish.display(primary_image)

100%|██████████| 1/1 [00:00<00:00, 18.14it/s]


In [ ]:
# bandpass filter to remove cellular background and camera noise
bandpass = starfish.image.Filter.Bandpass(lshort=.5, llong=7, threshold=0.0)
# gaussian blur to smooth z-axis
glp = starfish.image.Filter.GaussianLowPass(
    sigma=(1, 0, 0),
    is_volume=True
)

# pre-filter clip to remove low-intensity background signal
clip1 = starfish.image.Filter.Clip(p_min=50, p_max=100)
# post-filter clip to eliminate all but the highest-intensity peaks
clip2 = starfish.image.Filter.Clip(p_min=99, p_max=100, is_volume=True)

# spot detection
tlmpf = starfish.spots.DetectSpots.TrackpyLocalMaxPeakFinder(
    spot_diameter=5,  # must be odd integer
    min_mass=0.02,
    max_size=2,  # this is max radius
    separation=7,
    noise_size=0.65,  # this is not used because preprocess is False
    preprocess=False,
    percentile=10,  # this is irrelevant when min_mass, spot_diameter, and max_size are set properly
    verbose=True,
    is_volume=True,
)

for image_number, primary_image in enumerate(fov0.get_images(FieldOfView.PRIMARY_IMAGES)):
    print(f"Filtering image {image_number}...")
    filter_kwargs = dict(
        in_place=False,
        verbose=True,
        n_processes=None
    )
    clipped1 = clip1.run(primary_image, **filter_kwargs)
    clipped1.to_multipage_tiff('sample_data/first_clip_penk63_fov000.tiff')
    
    bandpassed = bandpass.run(clipped1, **filter_kwargs)
    bandpassed.to_multipage_tiff('sample_data/bandpassed_penk63_fov000.tiff')
    
    glped = glp.run(bandpassed, **filter_kwargs)
    glped.to_multipage_tiff('sample_data/lowpass_filtered_penk63_fov000.tiff')
    
    clipped2 = clip2.run(glped, **filter_kwargs)
    clipped2.to_multipage_tiff('sample_data/final_filtered_penk63_fov000.tiff')

In [8]:
print("Calling spots...")
spot_attributes = tlmpf.run(clipped2)
print(spot_attributes)

Calling spots...
<xarray.IntensityTable (features: 378, c: 1, r: 1)>
array([[[0.000962]],

       [[0.000975]],

       ...,

       [[0.000962]],

       [[0.00085 ]]])
Coordinates:
    eccentricity     (features) float64 nan nan nan nan nan ... nan nan nan nan
    ep               (features) float64 nan nan nan nan nan ... nan nan nan nan
    radius           (features) float64 1.476 1.474 1.471 ... 1.267 1.481 1.484
    raw_mass         (features) float64 0.0215 0.02176 ... 0.02233 0.02054
    total_intensity  (features) float64 0.02133 0.02152 ... 0.02211 0.02037
    x                (features) float64 331.1 306.0 269.1 ... 138.1 104.1
    y                (features) float64 909.0 989.0 ... 1.602e+03 1.621e+03
    z                (features) float64 8.01 7.999 8.971 ... 353.0 355.0 357.0
  * features         (features) int64 0 1 2 3 4 5 6 ... 372 373 374 375 376 377
  * c                (c) int64 0
  * r                (r) int64 0
    xc               (features) float64 1.617e-05 1

In [9]:
spot_attributes.to_netcdf('penk63_intensity_table.nc')

In [19]:
spot_attributes[0][0][0]

<xarray.IntensityTable (c: 1, r: 1)>
array([[0.000962]])
Coordinates:
    eccentricity     float64 nan
    ep               float64 nan
    radius           float64 1.476
    raw_mass         float64 0.0215
    total_intensity  float64 0.02133
    x                float64 331.1
    y                float64 909.0
    z                float64 8.01
    features         int64 0
  * c                (c) int64 0
  * r                (r) int64 0
    xc               float64 1.617e-05
    yc               float64 4.441e-05
    zc               float64 5e-05
Attributes:
    starfish:  {"log": [{"method": "Clip", "arguments": {"p_min": 50, "p_max"...